In [1]:
from owlready2 import *
print("Owlready2 is installed and ready to use!")

Owlready2 is installed and ready to use!


In [4]:
def analyze_ontology(ontology):
    print("\nAnalyzing ontology...")

    # Count instances
    instances = list(ontology.individuals())
    num_instances = len(instances)
    print(f"Number of instances: {num_instances}")

    # Iterate through each instance
    for instance in instances:
        print(f"\nInstance: {instance.name}")

        # Get all properties assigned to the instance
        properties = list(instance.get_properties())
        
        # Separate data properties and object properties
        data_properties = [prop for prop in properties if isinstance(prop, DataPropertyClass)]
        object_properties = [prop for prop in properties if isinstance(prop, ObjectPropertyClass)]

        # Data properties
        print(f"  Data properties assigned: {len(data_properties)}")
        for prop in data_properties:
            values = getattr(instance, prop.python_name, [])
            print(f"    - {prop.name}: {values}")

        # Object properties
        print(f"  Object properties assigned: {len(object_properties)}")
        for prop in object_properties:
            values = getattr(instance, prop.python_name, [])
            if values:
                related_names = [related.name for related in values]
                print(f"    - {prop.name}: {related_names}")
            else:
                print(f"    - {prop.name}: None")
            
def check_ontology(onto):
    print("\nChecking ontology consistency...")

    try:
        # Run the reasoner
        with onto:
            sync_reasoner_pellet() 

        # Check for inconsistent classes after instantiation of Circles use case
        inconsistent_classes = list(onto.inconsistent_classes())
        if inconsistent_classes:
            print("\nInconsistent classes found:")
            for cls in inconsistent_classes:
                # Print the class name or IRI
                print(f"Class: {cls.name}, IRI: {cls.iri}")
        else:
            print("\nThe ontology is consistent!")

    except OwlReadyInconsistentOntologyError as e:
        print("\nOntology is inconsistent. The reasoner reported an error:")
        print(e)

def load_ontology(ontology_path):
    # Load the ontology
    onto = get_ontology(ontology_path).load()

    # Display classes in the ontology
    print("Classes in the ontology:")
    for cls in onto.classes():
        print(cls)

    return onto

# Utility function to retrieve a class by name
def get_class_by_name(ontology, class_name):
        return next((cls for cls in ontology.classes() if cls.name == class_name), None)


def instantiate_ontology(onto):
        # Retrieve necessary classes
    DAOClass = get_class_by_name(onto, "DAO")
    RoleClass = get_class_by_name(onto, "Role")
    GovernanceAreaClass = get_class_by_name(onto, "Governance_Area")
    CommitteeClass = get_class_by_name(onto, "Committee")
    TokenTypeClass = get_class_by_name(onto, "token_type")
    TreasuryClass = get_class_by_name(onto, "DAO_treasury")
    TokenManagerClass = get_class_by_name(onto, "token_manager")
    TokenIssuanceClass = get_class_by_name(onto, "token_issuance")
    IncentiveClass = get_class_by_name(onto, "Incentive")
    RevenueManagementClass = get_class_by_name(onto, "revenue_management")
    ReactionRuleClass = get_class_by_name(onto, "Reaction_Rule")
    ConditionClass = get_class_by_name(onto, "Condition")
    ExecutiveActionClass = get_class_by_name(onto, "Executive_Action")
    VotePollingClass = get_class_by_name(onto, "vote_polling")
    VoteTallyingClass = get_class_by_name(onto, "vote_tallying")
    ProposalExecutionClass = get_class_by_name(onto, "proposal_execution")
    RoleAssignmentClass = get_class_by_name(onto, "Role_Assignment")

    # Error checking for all classes at once
    if not all([
        DAOClass, RoleClass, GovernanceAreaClass, CommitteeClass,
        TokenTypeClass, TreasuryClass, TokenManagerClass, TokenIssuanceClass,
        IncentiveClass, RevenueManagementClass, ReactionRuleClass, ConditionClass,
        ExecutiveActionClass, VotePollingClass, VoteTallyingClass,
        ProposalExecutionClass, RoleAssignmentClass
    ]):
        raise ValueError("One or more required classes are missing from the ontology!")

    # Create the CirclesDAO individual
    circles_dao = DAOClass("CirclesDAO")
    circles_dao.DAO_name.append("Circles DAO")
    circles_dao.mission_statement.append("Manage  the governance of a group in the Circles UBI system, its token economy and attest user contributions.")

    # Define roles

    group_member = RoleClass("GroupMember")
    group_member.role_description=[]
    group_member.role_description.append("Users depositing Circles tokens to join the group.")

    active_member = RoleClass("ActiveMember")
    active_member.role_description.append("Members contributing to the group governance.")

    treasury_manager = RoleClass("TreasuryManager")
    treasury_manager.role_description.append("Manages group funds.")

    general_assembly = CommitteeClass("GeneralAssembly")

    community_council = CommitteeClass("CommunityCouncil")
    community_council.n_agents.append(10)


    economic_council = CommitteeClass("EconomicCouncil")
    economic_council.n_agents.append(15)


    technical_council = CommitteeClass("TechnicalCouncil")
    technical_council.n_agents.append(7)


    # governance areas specification

    service_provision = GovernanceAreaClass("ServiceProvision")
    service_provision.description.append("Proposes and evaluates services for the Circles ecosystem.")
    service_provision.implementation.append("hybrid")

    treasury_management = GovernanceAreaClass("TreasuryManagement")
    treasury_management.description.append("Manages DAO funds.")
    treasury_management.implementation.append("onchain")

    group_membership = GovernanceAreaClass("GroupMembership")
    group_membership.description.append("Defines membership requirements.")
    group_membership.implementation.append("onchain")

    attestations = GovernanceAreaClass("Attestations")
    attestations.description.append("Attests user contributions.")
    attestations.implementation.append("onchain")
    
    emergency_response = GovernanceAreaClass("EmergencyResponse")
    emergency_response.description.append("Manages emergency situations.")
    emergency_response.implementation.append("hybrid")
    
    #  Token economy specification
    circles_token = TokenTypeClass("CirclesToken")
    circles_token.token_name.append("Circles Token")
    circles_token.fungibility.append("SFT")
    circles_token.transferable.append(True)
    circles_token.token_utility.extend(["financial", "governance"])

    group_currency = TokenTypeClass("GroupCurrency")
    group_currency.token_name.append("Group Currency")
    group_currency.fungibility.append("FT")
    group_currency.transferable.append(True)
    group_currency.token_utility.extend(["financial", "access"])

    perishable_share = TokenTypeClass("PerishableShare")
    perishable_share.token_name.append("Perishable Share")
    perishable_share.fungibility.append("SFT")
    perishable_share.transferable.append(True)
    perishable_share.token_utility.extend(["reputation", "governance"])

    stablecoin = TokenTypeClass("stablecoin")
    stablecoin.token_name.append("EUROE")
    stablecoin.fungibility.append("FT")
    stablecoin.transferable.append(True)
    stablecoin.token_utility.append("financial")

    # Token Manager
    token_manager = TokenManagerClass("DAOTokenManager")
    token_manager.token_economy_goal.append("Manage and issue DAO tokens for governance and incentives")
    token_manager.manages = [circles_token, group_currency, perishable_share]

    # DAO Treasury
    dao_treasury = TreasuryClass("DAOTreasury")
    dao_treasury.fiscal_policy_goal.append("Store and manage EUROE tokens for DAO operations")
    dao_treasury.stores = [stablecoin]

    # Token Issuance Mechanism for Perishable Share
    issuance_condition_ps = ConditionClass("IssuanceConditionPerishableShare")
    issuance_condition_ps.description.append("expiry date elapsed")

    issuance_rule_ps = ReactionRuleClass("IssuanceRulePerishableShare")
    issuance_rule_ps.description.append("periodically burn Perishable Shares")
    issuance_rule_ps.has_rule_condition = [issuance_condition_ps]

    issuance_action_ps = ExecutiveActionClass("IssuanceActionPerishableShare")
    issuance_action_ps.description.append("Burn Perishable Shares")
    issuance_rule_ps.triggers = [issuance_action_ps]

    issuance_mechanism_ps = TokenIssuanceClass("PerishableShareIssuance")
    issuance_mechanism_ps.implements = [issuance_rule_ps]
    perishable_share.implements = [issuance_mechanism_ps]

    # Incentive Mechanism for Perishable Share
    incentive_condition_ps = ConditionClass("IncentiveConditionPerishableShare")
    incentive_condition_ps.description.append("User contribution verified")

    incentive_rule_ps = ReactionRuleClass("IncentiveRulePerishableShare")
    incentive_rule_ps.description.append("Reward with Perishable Shares active users when contribution is verified")
    incentive_rule_ps.has_rule_condition = [incentive_condition_ps]

    incentive_action_ps = ExecutiveActionClass("IncentiveActionPerishableShare")
    incentive_action_ps.description.append("issue and distribute Perishable Shares to active user")
    incentive_rule_ps.triggers = [incentive_action_ps]

    incentive_mechanism_ps = IncentiveClass("PerishableShareIncentive")
    incentive_mechanism_ps.implements = [incentive_rule_ps]
    perishable_share.implements.append(incentive_mechanism_ps)

    # Define Token Issuance Mechanism for Group Currency
    issuance_condition_gc = ConditionClass("IssuanceConditionGroupCurrency")
    issuance_condition_gc.description.append("Circles token staking verified")
    
    issuance_rule_gc = ReactionRuleClass("IssuanceRuleGroupCurrency")
    issuance_rule_gc.description.append("Mint new Group Currency tokens when Circles tokens are staked")
    issuance_rule_gc.has_rule_condition = [issuance_condition_gc]

    issuance_action_gc = ExecutiveActionClass("IssuanceActionGroupCurrency")
    issuance_action_gc.description.append("Mint and distribute Group Currency tokens to staker")
    issuance_rule_gc.triggers = [issuance_action_gc]

    issuance_mechanism_gc = TokenIssuanceClass("GroupCurrencyIssuance")
    issuance_mechanism_gc.implements = [issuance_rule_gc]
    group_currency.implements = [issuance_mechanism_gc]

    # Define Revenue Management Mechanism for Circles Token
    revenue_condition_ct = ConditionClass("RevenueConditionCirclesToken")
    revenue_condition_ct.description.append("Funding request for community projects approval")

    revenue_rule_ct = ReactionRuleClass("RevenueRuleCirclesToken")
    revenue_rule_ct.description.append("lend Circles Tokens to fund community projects")
    revenue_rule_ct.has_rule_condition = [revenue_condition_ct]

    revenue_action_ct = ExecutiveActionClass("RevenueActionCirclesToken")
    revenue_action_ct.description.append("Transfer Circles Tokens to fund community projects")
    revenue_rule_ct.triggers = [revenue_action_ct]

    revenue_mechanism_ct = RevenueManagementClass("CirclesRevenueManagement")
    revenue_mechanism_ct.implements = [revenue_rule_ct]
    circles_token.implements = [revenue_mechanism_ct]

    # Define Revenue Management Mechanism for EUROE
    revenue_condition_euroe = ConditionClass("RevenueConditionEUROE")
    revenue_condition_euroe.description.append("Revenue allocation triggered")

    revenue_rule_euroe = ReactionRuleClass("RevenueRuleEUROE")
    revenue_rule_euroe.description.append("Allocate revenue from EUROE token holdings")
    revenue_rule_euroe.has_rule_condition = [revenue_condition_euroe]

    revenue_action_euroe = ExecutiveActionClass("RevenueActionEUROE")
    revenue_action_euroe.description.append("Allocate EUROE revenue to DAO initiatives")
    revenue_rule_euroe.triggers = [revenue_action_euroe]

    revenue_mechanism_euroe = RevenueManagementClass("EUROERevenueManagement")
    revenue_mechanism_euroe.implements = [revenue_rule_euroe]
    stablecoin.implements = [revenue_mechanism_euroe]

    # Define Exchange Mechanism: Swap EUROE <-> GroupCurrency
    exchange_condition = ConditionClass("ExchangeCondition")
    exchange_condition.description.append("Token transfer detected (EUROE or GroupCurrency)")

    exchange_rule = ReactionRuleClass("ExchangeRule")
    exchange_rule.description.append("Swap EUROE for GroupCurrency or vice versa based on exchange rate")
    exchange_rule.has_rule_condition = [exchange_condition]

    exchange_action = ExecutiveActionClass("ExchangeAction")
    exchange_action.description.append("Execute token swap based on defined exchange rate")
    exchange_rule.triggers = [exchange_action]

    exchange_mechanism = RevenueManagementClass("ExchangeMechanism")
    exchange_mechanism.implements = [exchange_rule]
    dao_treasury.implements.append(exchange_mechanism)

    # Define Incentive Mechanism: Distribute GroupCurrency to service providers
    incentive_condition_service = ConditionClass("IncentiveConditionService")
    incentive_condition_service.description.append("Proof of service completion submitted")

    incentive_rule_service = ReactionRuleClass("IncentiveRuleService")
    incentive_rule_service.description.append("Distribute GroupCurrency to service providers upon proof of service completion")
    incentive_rule_service.has_rule_condition = [incentive_condition_service]

    incentive_action_service = ExecutiveActionClass("IncentiveActionService")
    incentive_action_service.description.append("Transfer GroupCurrency to the service provider’s account")
    incentive_rule_service.triggers = [incentive_action_service]

    incentive_mechanism_service = IncentiveClass("ServiceIncentiveMechanism")
    incentive_mechanism_service.implements = [incentive_rule_service]
    dao_treasury.implements.append(incentive_mechanism_service)
    
    
    # Add vote polling mechanism
    vote_polling = VotePollingClass("VotePollingMechanism")
    vote_polling.ballot_type.append("single")
    vote_polling.proposal_submission.append("continuous")
    vote_polling.alternative_selection.append("binary")

    # Define condition and reaction rule for vote polling
    polling_condition = ConditionClass("PollingCondition")
    polling_condition.description.append("Proposal must be submitted")

    polling_reaction_rule = ReactionRuleClass("PollingReactionRule")
    polling_reaction_rule.description.append("Trigger rule when proposal is cast by participants")
    polling_reaction_rule.has_rule_condition = [polling_condition]

    # Define executive action for vote polling
    polling_action = ExecutiveActionClass("PollingAction")
    polling_action.description.append("Record the votes cast during polling")
    polling_reaction_rule.triggers = [polling_action]

    # Assign reaction rule and action to vote polling
    vote_polling.implements = [polling_reaction_rule]

    # Add vote tallying mechanisms
    vote_tallying_SM = VoteTallyingClass("SimpleMajorityTallyingMechanism")
    vote_tallying_SM.vote_weight.append("uniform")
    vote_tallying_SM.token_ref_ID.append("GroupCurrency")
    vote_tallying_SM.participant_quorum.append(40)

    tallying_condition_SM = ConditionClass("TallyingConditionSM")
    tallying_condition_SM.description.append("Polling must be complete for Simple Majority")

    tallying_reaction_rule_SM = ReactionRuleClass("TallyingReactionRuleSM")
    tallying_reaction_rule_SM.description.append("Trigger when proposal participation quorum is met (Simple Majority)")
    tallying_reaction_rule_SM.has_rule_condition = [tallying_condition_SM]

    # Define executive action for simple majority tallying
    tallying_action_SM = ExecutiveActionClass("TallyingActionSM")
    tallying_action_SM.description.append("Calculate votes using simple majority method")
    tallying_reaction_rule_SM.triggers = [tallying_action_SM]

    vote_tallying_SM.implements = [tallying_reaction_rule_SM]

    vote_tallying_OG = VoteTallyingClass("OptimisticGovernanceTallyingMechanism")
    vote_tallying_OG.vote_weight.append("token_balance")
    vote_tallying_OG.token_ref_ID.append("PerishableShare")
    vote_tallying_OG.participant_quorum.append(30)
    vote_tallying_OG.majority_quorum.append(60)

    tallying_condition_OG = ConditionClass("TallyingConditionOG")
    tallying_condition_OG.description.append("Polling must be complete for Optimistic Governance")

    tallying_reaction_rule_OG = ReactionRuleClass("TallyingReactionRuleOG")
    tallying_reaction_rule_OG.description.append("Trigger when challenge period elapsed (Optimistic Governance)")
    tallying_reaction_rule_OG.has_rule_condition = [tallying_condition_OG]

    # Define executive action for optimistic governance tallying
    tallying_action_OG = ExecutiveActionClass("TallyingActionOG")
    tallying_action_OG.description.append("Calculate challenges for optimistic governance protocol")
    tallying_reaction_rule_OG.triggers = [tallying_action_OG]

    vote_tallying_OG.implements = [tallying_reaction_rule_OG]

    # Add proposal execution mechanisms
    proposal_execution_general = ProposalExecutionClass("ProposalExecutionGeneral")
    proposal_execution_general.autonomous_execution.append(False)

    execution_condition_general = ConditionClass("ExecutionConditionGeneral")
    execution_condition_general.description.append("Proposal must not be challenged by General Assembly Voting")

    execution_reaction_rule_general = ReactionRuleClass("ExecutionRuleGeneral")
    execution_reaction_rule_general.description.append("Trigger rule for time elapsed")
    execution_reaction_rule_general.has_rule_condition = [execution_condition_general]

    # Define executive action for general proposal execution
    execution_action_general = ExecutiveActionClass("ExecutionActionGeneral")
    execution_action_general.description.append("Execute the proposal not opposed by General Assembly")
    execution_reaction_rule_general.triggers = [execution_action_general]

    proposal_execution_general.implements = [execution_reaction_rule_general]

    proposal_execution_council = ProposalExecutionClass("ProposalExecutionCouncil")
    proposal_execution_council.autonomous_execution.append(True)

    execution_condition_council = ConditionClass("ExecutionConditionCouncil")
    execution_condition_council.description.append("Proposal must be approved by Council Voting")

    execution_reaction_rule_council = ReactionRuleClass("ExecutionRuleCouncil")
    execution_reaction_rule_council.description.append("Trigger rule for proposal approval by council")
    execution_reaction_rule_council.has_rule_condition = [execution_condition_council]

    # Define executive action for council proposal execution
    execution_action_council = ExecutiveActionClass("ExecutionActionCouncil")
    execution_action_council.description.append("Execute the proposal approved by Council Voting")
    execution_reaction_rule_council.triggers = [execution_action_council]

    proposal_execution_council.implements = [execution_reaction_rule_council]

    # Define object property consists_of to link policies to mechanisms
    voting_protocol_general = get_class_by_name(onto, "decision_making_policy")("GeneralAssemblyVoting")
    voting_protocol_general.consists_of = [vote_polling, vote_tallying_OG, proposal_execution_general]

    voting_protocol_council = get_class_by_name(onto, "decision_making_policy")("CouncilVoting")
    voting_protocol_council.consists_of = [vote_polling, vote_tallying_SM, proposal_execution_council]
    
    # Assign DAO Membership
    service_provision_role = RoleAssignmentClass("ServiceProvisionRole")
    service_provision_role.role_ID.append("GeneralAssemblyMembership")
    service_provision_role.assignment_method.append("Group Currency ownership")

    membership_management = RoleAssignmentClass("CouncilMembershipManagement")
    membership_management.role_ID.append("CouncilMembershipManagement")
    membership_management.assignment_method.append("Perishable Share ownership")
    
    # Save the ontology
    onto.save(file="Circles_Ubi_Instantiated.owl", format="rdfxml")
    print("Ontology instantiated and saved as 'Circles_Ubi_Instantiated.owl'.")
    


In [5]:
ontology_path = "/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/DAOMod_Ontology.rdf"
onto = load_ontology(ontology_path)
check_ontology(onto)
instantiate_ontology(onto)
check_ontology(onto)
analyze_ontology(onto)

Classes in the ontology:
eSourcing_Ontology.Contract
DAOMod_Ontology.Actor
eSourcing_Ontology.Reaction_Rule
eSourcing_Ontology.Copier_Action
eSourcing_Ontology.Enabler_Action
eSourcing_Ontology.Executive_Action
eSourcing_Ontology.Link_Property
eSourcing_Ontology.Sink
eSourcing_Ontology.Passing_Type
eSourcing_Ontology.Block
eSourcing_Ontology.Aim_Case
eSourcing_Ontology.Scope
eSourcing_Ontology.Sourcing_Sphere
eSourcing_Ontology.Aim_Task
eSourcing_Ontology.Document
eSourcing_Ontology.Var_Section
eSourcing_Ontology.Privilege
eSourcing_Ontology.State_Constraint_Rule
eSourcing_Ontology.Orgnizational_Position
eSourcing_Ontology.Appointment
eSourcing_Ontology.Task
eSourcing_Ontology.Role
eSourcing_Ontology.Mediator
eSourcing_Ontology.Party
eSourcing_Ontology.Rule_Section
eSourcing_Ontology.Snippet_Section
eSourcing_Ontology.Process
eSourcing_Ontology.Process_Section
eSourcing_Ontology.Role_Delegation
eSourcing_Ontology.Cancelled
eSourcing_Ontology.Terminated
eSourcing_Ontology.Unfulfilled
eS

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/log4j-core-2.19.0.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology/myenv/lib/python3.12/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/home/sowelo/Scrivania/DAOMod/DAOMod/DAOMod Ontology


The ontology is consistent!
Ontology instantiated and saved as 'Circles_Ubi_Instantiated.owl'.

Checking ontology consistency...

The ontology is consistent!

Analyzing ontology...
Number of instances: 71

Instance: CirclesDAO
  Data properties assigned: 2
    - mission_statement: ['Manage  the governance of a group in the Circles UBI system, its token economy and attest user contributions.', 'Manage  the governance of a group in the Circles UBI system, its token economy and attest user contributions.']
    - DAO_name: ['Circles DAO', 'Circles DAO']
  Object properties assigned: 0

Instance: GroupMember
  Data properties assigned: 1
    - role_description: ['Users depositing Circles tokens to join the group.']
  Object properties assigned: 0

Instance: ActiveMember
  Data properties assigned: 1
    - role_description: ['Members contributing to the group governance.', 'Members contributing to the group governance.']
  Object properties assigned: 0

Instance: TreasuryManager
  Data prop

* Owlready2 * Pellet took 1.6990156173706055 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)
